In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

In [3]:
sys.path.append("/home/caleml/main-pe/")

In [4]:
from data.datasets.h36m import Human36M
from data.loader import BatchLoader
from data.utils import transform, camera
from data.utils.data_utils import TEST_MODE, TRAIN_MODE, VALID_MODE

from model import config, measures, callbacks
from model.networks.multi_branch_model import MultiBranchModel
from model.utils import pose_format, log

In [5]:
from experiments.common import exp_init

In [ ]:
'''
Problèmes save_model
-> save_model error -> tester avec une version supérieure de TF / api keras intégrée
-> load_weights error -> tester avec le name rreur numpy arrays au loading
-> save_model errr a la mobilenet -> essayer de mieux nommer les branches ?

For join the models in one, only you need change the line 60 in frontend.py
(features = self.feature_extractor.extract(input_image) to
features = self.feature_extractor.feature_extractor.output)
and the line 71
(self.model = Model([input_image, self.true_boxes], output) to
self.model = Model([self.feature_extractor.feature_extractor.input , self.true_boxes], output))

https://github.com/experiencor/keras-yolo2/issues/358
https://github.com/keras-team/keras/issues/10784
'''

## Small training with the save callback

In [ ]:
conf = {
    'exp_type': 'hybrid_SAVE_TEST_NB',
    'dim': 3,
    'n_joints': 17,
    'pose_blocks': 2,
    'dataset_name': 'h36m',
    'batch_size': 8,
    'n_epochs': 60
}

In [ ]:
model_folder = exp_init(conf)

In [ ]:
# local loading
h36m_path = '/home/caleml/datasets/h36m'
h36m = Human36M(h36m_path, dataconf=config.human36m_dataconf, poselayout=pose_format.pa17j3d, topology='frames') 

In [ ]:
# classical i_hat + pose format
data_tr_h36m = BatchLoader(
        h36m, 
        ['frame'], 
        ['frame'] + ['pose'] * conf['pose_blocks'],
        TRAIN_MODE, 
        batch_size=conf['batch_size'],
        shuffle=True)

In [ ]:
model = MultiBranchModel(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
model.build()

In [ ]:
from model import callbacks
cb_list = []
cb_list.append(callbacks.SaveModel(model_folder))

In [ ]:
model.train(data_tr_h36m, steps_per_epoch=10, model_folder=model_folder, n_epochs=2, cb_list=cb_list)

## Loading

In [ ]:
weights_path = '/home/caleml/pe_experiments/exp_20190506_1632_hybrid_SAVE_TEST_NB__2b_bs8/weights_001.h5'

In [ ]:
from model.networks.multi_branch_model import MultiBranchModel
eval_model = MultiBranchModel(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
eval_model.load_weights(weights_path, pose_only=False, by_name=True)

## 2D model

In [14]:
conf_2d = {
    'exp_type': '2d_SAVE_TEST_NB',
    'dim': 2,
    'n_joints': 16,
    'pose_blocks': 2,
    'dataset_name': 'mpii',
    'batch_size': 16,
    'n_epochs': 60
}

In [7]:
model_folder = exp_init(conf_2d)

Conducting experiment for 60 epochs and 2 blocks in folder /home/caleml/pe_experiments/exp_20190513_1248_2d_SAVE_TEST_NB__2b_bs32


In [8]:
from data.datasets.mpii import MpiiSinglePerson

In [9]:
mpii_path = '/home/caleml/datasets/mpii'
mpii = MpiiSinglePerson(mpii_path, dataconf=config.mpii_dataconf)

In [15]:
data_tr_mpii = BatchLoader(mpii,
                           ['frame'],
                           ['frame'] + ['pose'] * conf_2d['pose_blocks'],
                           TRAIN_MODE,
                           batch_size=conf_2d['batch_size'],
                           shuffle=False)

y_dict ['frame', 'pose', 'pose'], allkeys ['frame', 'frame', 'pose', 'pose']


In [11]:
from model.networks.multi_branch_model import MultiBranchModel
model = MultiBranchModel(dim=conf_2d['dim'], n_joints=conf_2d['n_joints'], nb_pose_blocks=conf_2d['pose_blocks'])
model.build()

/home/caleml/.local/lib/python3.4/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


zp_depth 1024
pose shape (?, 16, 2), vis shape (?, 16, 1), concat shape (?, 16, 3)
pose shape (?, 16, 2), vis shape (?, 16, 1), concat shape (?, 16, 3)
Last H shape Tensor("fReMap2/batch_normalization_16/cond/Merge:0", shape=(?, 32, 32, 576), dtype=float32)
Build E_a 10.419375896453857, build E_p 7.103278160095215, decoder D 0.7753181457519531
Input shape (?, 256, 256, 3)
Shape z_a (?, 16, 16, 1024), shape z_p (?, 16, 16, 1024)
Outputs shape [(None, 256, 256, 3), (None, 16, 3), (None, 16, 3)]
rec y_pred shape (?, 256, 256, 3)
Final model summary
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_input (InputLayer)        (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
appearance_model (Model)        (None, 16, 16, 1024)

In [12]:
from model import callbacks
cb_list = []
cb_list.append(callbacks.SaveModel(model_folder))

In [16]:
model.train(data_tr_mpii, steps_per_epoch=10, model_folder=model_folder, n_epochs=2, cb_list=cb_list)

Training with 1 callbacks
Epoch 1/2
 9/10 [==========================>...] - ETA: 1s - loss: 38.4976 - decoder_loss: 32.0965 - pose_model_loss: 3.3439
Saving model @epoch=001 to /home/caleml/pe_experiments/exp_20190513_1248_2d_SAVE_TEST_NB__2b_bs32/weights_001.h5
10/10 [==============================] - 12s 1s/step - loss: 38.0321 - decoder_loss: 31.6586 - pose_model_loss: 3.3524
Epoch 2/2
 9/10 [==========================>...] - ETA: 0s - loss: 24.5566 - decoder_loss: 19.2855 - pose_model_loss: 2.9162
Saving model @epoch=002 to /home/caleml/pe_experiments/exp_20190513_1248_2d_SAVE_TEST_NB__2b_bs32/weights_002.h5
10/10 [==============================] - 5s 541ms/step - loss: 23.9735 - decoder_loss: 18.7168 - pose_model_loss: 2.8907


### Loading 2D

In [17]:
weights_path = '/home/caleml/pe_experiments/exp_20190513_1248_2d_SAVE_TEST_NB__2b_bs32/weights_001.h5'

In [18]:
from model.networks.multi_branch_model import MultiBranchModel
eval_model = MultiBranchModel(dim=conf_2d['dim'], n_joints=conf_2d['n_joints'], nb_pose_blocks=conf_2d['pose_blocks'])
eval_model.load_weights(weights_path, pose_only=False, by_name=False)

/home/caleml/.local/lib/python3.4/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


zp_depth 1024
pose shape (?, 16, 2), vis shape (?, 16, 1), concat shape (?, 16, 3)
pose shape (?, 16, 2), vis shape (?, 16, 1), concat shape (?, 16, 3)
Last H shape Tensor("fReMap2_1/batch_normalization_43/cond/Merge:0", shape=(?, 32, 32, 576), dtype=float32)
Build E_a 11.927100419998169, build E_p 22.101194381713867, decoder D 0.7576556205749512
Input shape (?, 256, 256, 3)
Shape z_a (?, 16, 16, 1024), shape z_p (?, 16, 16, 1024)
Outputs shape [(None, 256, 256, 3), (None, 16, 3), (None, 16, 3)]
rec y_pred shape (?, 256, 256, 3)
Final model summary
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_input (InputLayer)        (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
appearance_model (Model)        (None, 16, 16, 10

ValueError: axes don't match array